# Data Prepare

Outputs: two dataframes - edge and node specific data sets.





In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import geoplot
import pickle
import geoplot.crs as gcrs

/home/jasonllu/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.9.0-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
import statsmodels.api as sm

In [7]:
mount_path = "/mnt/c/Users/jason/Dropbox (MIT)/"

In [8]:
# read dfs
trans_mcc_adelaide_df = pd.read_pickle("../../data_process/trans_mcc_adelaide_df.pkl")
trans_age_adelaide_df = pd.read_pickle("../../data_process/trans_age_adelaide_df.pkl")
flow_adelaide_df = pd.read_pickle("../../data_process/flow_adelaide_df.pkl")

# read spatial files
sa2_south_au = gpd.read_file("../../data_process/shapefiles/sa2_south_au.shp")
sa2_adelaide = gpd.read_file('../../data_process/shapefiles/sa2_adelaide.shp')

# read road networks
sa2_roads = gpd.read_file("../../data_process/shapefiles/sa2_roads.shp")

# read job and income data
jobs_all=pd.read_pickle("../../data_process/jobs_all.pkl")
jobs_industries=pd.read_pickle("../../data_process/jobs_industries.pkl")

In [9]:
# read google api info
with open(mount_path + '/SA data/dataSA/OD_Google_API_raw.pickle', 'rb') as w:
    OD_google_raw = pickle.load(w)

with open(mount_path + '/SA data/dataSA/OD_Google_API_With_Map_Info.pickle', 'rb') as w:
    OD_google_with_map = pickle.load(w)


In [15]:
# read OD dictionary
with open("../../data_process/OD_full_path_dic.pickle", 'rb') as f:
    OD_full_path=pickle.load(f)

# read road class shp
sa2_adelaide_road_proj = gpd.read_file("../../data_process/shapefiles/sa2_adelaide_proj_road_class.shp")


In [16]:
# 
sa2_adelaide_road_proj

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,...,class_BUS,class_COLL,class_FREE,class_HWY,class_LOCL,class_SUBA,class_TRK2,class_TRK4,class_UND,geometry
0,401011001,41001,Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,13.0,402.0,0.0,0.0,1089.0,501.0,0.0,0.0,10.0,"POLYGON ((417263.891 -3939518.653, 417262.345 ..."
1,401011002,41002,North Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,3.0,221.0,0.0,0.0,248.0,108.0,0.0,0.0,1.0,"POLYGON ((417476.199 -3936647.849, 417544.905 ..."
2,401021003,41003,Adelaide Hills,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,0.0,244.0,0.0,0.0,1096.0,188.0,2737.0,37.0,0.0,"POLYGON ((433666.133 -3928044.498, 433628.032 ..."
3,401021004,41004,Aldgate - Stirling,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,0.0,107.0,88.0,0.0,1360.0,421.0,700.0,0.0,19.0,"POLYGON ((427353.316 -3946292.628, 427421.744 ..."
4,401021005,41005,Hahndorf - Echunga,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,0.0,49.0,24.0,0.0,438.0,73.0,423.0,0.0,0.0,"POLYGON ((435785.372 -3950672.079, 435799.037 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,404031105,41105,Fulham,40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,0.0,32.0,0.0,0.0,124.0,32.0,0.0,0.0,0.0,"POLYGON ((410502.489 -3938804.420, 410496.375 ..."
106,404031106,41106,Lockleys,40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,0.0,161.0,0.0,0.0,379.0,107.0,0.0,0.0,1.0,"POLYGON ((411842.413 -3938239.458, 411857.841 ..."
107,404031107,41107,Plympton,40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,0.0,323.0,0.0,45.0,780.0,57.0,0.0,0.0,0.0,"POLYGON ((413778.862 -3941759.801, 413777.655 ..."
108,404031108,41108,Richmond (SA),40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,0.0,261.0,0.0,94.0,648.0,136.0,0.0,0.0,3.0,"POLYGON ((414829.341 -3937727.856, 414828.056 ..."


In [17]:
jobs_all.columns

Index(['sa2_name16', 'num_jobs_000_persons',
       'median_income_per_job_aud_persons',
       'median_income_per_job_aud_females', 'median_income_per_job_aud_males',
       'sa2_code16', 'num_jobs_000_males', 'num_jobs_000_females'],
      dtype='object')

In [18]:
jobs_all.shape

(172, 8)

In [19]:
jobs_all.head(10)

,sa2_name16,num_jobs_000_persons,median_income_per_job_aud_persons,median_income_per_job_aud_females,median_income_per_job_aud_males,sa2_code16,num_jobs_000_males,num_jobs_000_females
0,Seaford (SA),16.014,27786.0,23253.0,34636.0,403041087,8.125,7.890
1,Nairne,3.864,27429.0,20528.0,37708.0,401021009,1.912,1.947
2,Uraidla - Summertown,4.162,27341.0,21593.0,36788.0,401021010,2.060,2.098
3,West Beach,3.909,28657.0,23029.0,36587.0,404031109,1.894,2.015
4,Barossa - Angaston,4.709,21687.0,17078.0,28462.0,405011110,2.444,2.270
5,Light,6.505,25328.0,20509.0,32376.0,405011111,3.469,3.031
6,Lyndoch,4.459,27739.0,20396.0,37630.0,405011112,2.328,2.135
7,Mallala,2.871,20552.0,16497.0,24218.0,405011113,1.518,1.349
8,Nuriootpa,4.842,25876.0,20724.0,33849.0,405011114,2.608,2.232
9,Port Pirie Region,1.879,24843.0,21261.0,29969.0,405031123,1.004,0.877


In [20]:
# 
jobs_all_sub = jobs_all[['num_jobs_000_persons', 'sa2_code16', 'median_income_per_job_aud_persons']]
jobs_all_sub

,num_jobs_000_persons,sa2_code16,median_income_per_job_aud_persons
0,16.014,403041087,27786.0
1,3.864,401021009,27429.0
2,4.162,401021010,27341.0
3,3.909,404031109,28657.0
4,4.709,405011110,21687.0
...,...,...,...
167,2.605,407031166,20087.0
168,4.607,407031167,11122.0
169,3.988,407031168,12606.0
170,3.773,407031169,16368.0


In [21]:
# 
flow_adelaide_df.columns

Index(['timestamp', 'sa2', 'state', 'agent_home_sa2', 'unique_agents',
       'sum_stay_duration', 'total_stays'],
      dtype='object')

In [22]:
# 
sa2_adelaide_road_proj.head()

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,...,class_BUS,class_COLL,class_FREE,class_HWY,class_LOCL,class_SUBA,class_TRK2,class_TRK4,class_UND,geometry
0,401011001,41001,Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,13.0,402.0,0.0,0.0,1089.0,501.0,0.0,0.0,10.0,"POLYGON ((417263.891 -3939518.653, 417262.345 ..."
1,401011002,41002,North Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,3.0,221.0,0.0,0.0,248.0,108.0,0.0,0.0,1.0,"POLYGON ((417476.199 -3936647.849, 417544.905 ..."
2,401021003,41003,Adelaide Hills,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,0.0,244.0,0.0,0.0,1096.0,188.0,2737.0,37.0,0.0,"POLYGON ((433666.133 -3928044.498, 433628.032 ..."
3,401021004,41004,Aldgate - Stirling,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,0.0,107.0,88.0,0.0,1360.0,421.0,700.0,0.0,19.0,"POLYGON ((427353.316 -3946292.628, 427421.744 ..."
4,401021005,41005,Hahndorf - Echunga,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,0.0,49.0,24.0,0.0,438.0,73.0,423.0,0.0,0.0,"POLYGON ((435785.372 -3950672.079, 435799.037 ..."


# Prepare Network Data

In [23]:
# replace col names
flow_adelaide_df.rename(columns={'agent_home_sa2':'origin','sa2':'destination'}, inplace=True)
flow_adelaide_df


,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays
1716,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217
1717,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322
1718,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334
1719,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487
1720,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278
...,...,...,...,...,...,...,...
15630,2018-02-01T00:00:00.000+11:00,404031105,4,404031109,2304,5876916,12172
15631,2018-02-01T00:00:00.000+11:00,404031106,4,404031109,2967,5413932,16935
15632,2018-02-01T00:00:00.000+11:00,404031107,4,404031109,1811,937583,8366
15633,2018-02-01T00:00:00.000+11:00,404031108,4,404031109,2464,1334692,11329


In [24]:
# init
flow_adelaide_df['OD'] = ''
flow_adelaide_df['OD'] = flow_adelaide_df['origin'] + flow_adelaide_df['destination']
flow_adelaide_df.groupby(by='OD').sum() # no repetition. 


,state,unique_agents,sum_stay_duration,total_stays
OD,,,,
401011001401011001,4,19863,268779414,681217
401011001401011002,4,6911,4857456,29322
401011001401021003,4,404,253764,1334
401011001401021004,4,1184,903377,5487
401011001401021005,4,700,189721,1278
...,...,...,...,...
404031109404031105,4,2304,5876916,12172
404031109404031106,4,2967,5413932,16935
404031109404031107,4,1811,937583,8366


In [25]:
# reindex
flow_adelaide_df.index = np.arange(flow_adelaide_df.shape[0])
flow_adelaide_df

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002
2,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334,401011001401021003
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004
4,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278,401011001401021005
...,...,...,...,...,...,...,...,...
10668,2018-02-01T00:00:00.000+11:00,404031105,4,404031109,2304,5876916,12172,404031109404031105
10669,2018-02-01T00:00:00.000+11:00,404031106,4,404031109,2967,5413932,16935,404031109404031106
10670,2018-02-01T00:00:00.000+11:00,404031107,4,404031109,1811,937583,8366,404031109404031107
10671,2018-02-01T00:00:00.000+11:00,404031108,4,404031109,2464,1334692,11329,404031109404031108


In [26]:
# create ten columns here.
road_attribute_names_list = ['class_ART', 'class_BUS', 'class_COLL',
                             'class_FREE', 'class_HWY', 'class_LOCL', 'class_SUBA', 'class_TRK2',
                             'class_TRK4', 'class_UND']
flow_adelaide_df[road_attribute_names_list] = 0.0


In [27]:
# add the road attributes on the shortest path to the flow_adelaide_df.
# time cost: 3-5 mins?
for idx in np.arange(flow_adelaide_df.shape[0]):
    origin = flow_adelaide_df.loc[idx, 'origin']
    destination = flow_adelaide_df.loc[idx, 'destination']
    o_idx = sa2_adelaide.index[sa2_adelaide.SA2_MAIN16==origin].tolist()[0]
    d_idx = sa2_adelaide.index[sa2_adelaide.SA2_MAIN16==destination].tolist()[0]
#     print(o_idx,d_idx)
    
    try:
        # OD_full_path might not have all the shortest path...
        idx_list_on_shortest_path = OD_full_path[(o_idx, d_idx)]
        for node_on_shortest_path in idx_list_on_shortest_path:
            flow_adelaide_df.loc[idx, road_attribute_names_list] += sa2_adelaide_road_proj.loc[node_on_shortest_path, road_attribute_names_list]        
    except KeyError as error:
        pass


In [28]:
flow_adelaide_df.shape

(10673, 18)

In [29]:
flow_adelaide_df.head(20)

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD,class_ART,class_BUS,class_COLL,class_FREE,class_HWY,class_LOCL,class_SUBA,class_TRK2,class_TRK4,class_UND
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001,208.0,13.0,402.0,0.0,0.0,1089.0,501.0,0.0,0.0,10.0
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002,267.0,16.0,623.0,0.0,0.0,1337.0,609.0,0.0,0.0,11.0
2,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334,401011001401021003,838.0,82.0,1393.0,0.0,0.0,4316.0,947.0,2786.0,37.0,14.0
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004,430.0,13.0,866.0,106.0,3.0,3357.0,1034.0,747.0,0.0,33.0
4,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278,401011001401021005,517.0,13.0,915.0,130.0,3.0,3795.0,1107.0,1170.0,0.0,33.0
5,2018-02-01T00:00:00.000+11:00,401021006,4,401011001,508,344551,1994,401011001401021006,361.0,13.0,1090.0,1.0,0.0,3746.0,1252.0,1728.0,1.0,22.0
6,2018-02-01T00:00:00.000+11:00,401021007,4,401011001,460,636111,3452,401011001401021007,583.0,13.0,1122.0,159.0,3.0,5144.0,1253.0,1297.0,0.0,57.0
7,2018-02-01T00:00:00.000+11:00,401021008,4,401011001,199,83042,583,401011001401021008,618.0,13.0,993.0,154.0,3.0,4809.0,1349.0,2135.0,0.0,33.0
8,2018-02-01T00:00:00.000+11:00,401021009,4,401011001,103,143591,543,401011001401021009,361.0,13.0,1124.0,7.0,0.0,4007.0,1335.0,1773.0,1.0,26.0
9,2018-02-01T00:00:00.000+11:00,401021010,4,401011001,685,347208,1897,401011001401021010,263.0,13.0,796.0,0.0,0.0,2876.0,1025.0,922.0,0.0,15.0


In [30]:
# add the job information to flow dataframe.
# origin
flow_adelaide_df=flow_adelaide_df.merge(jobs_all_sub, left_on='origin', right_on='sa2_code16', how = 'left')
flow_adelaide_df=flow_adelaide_df.rename(columns={'num_jobs_000_persons':'num_jobs_000_persons_origin', 'median_income_per_job_aud_persons':'median_income_per_job_aud_origin'})

# destination
flow_adelaide_df=flow_adelaide_df.merge(jobs_all_sub, left_on='destination', right_on='sa2_code16', how = 'left')
flow_adelaide_df=flow_adelaide_df.rename(columns={'num_jobs_000_persons':'num_jobs_000_persons_destination', 'median_income_per_job_aud_persons':'median_income_per_job_aud_destination'})

flow_adelaide_df

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD,class_ART,class_BUS,...,class_SUBA,class_TRK2,class_TRK4,class_UND,num_jobs_000_persons_origin,sa2_code16_x,median_income_per_job_aud_origin,num_jobs_000_persons_destination,sa2_code16_y,median_income_per_job_aud_destination
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001,208.0,13.0,...,501.0,0.0,0.0,10.0,14.057,401011001,17265.0,14.057,401011001,17265.0
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002,267.0,16.0,...,609.0,0.0,0.0,11.0,14.057,401011001,17265.0,5.459,401011002,24773.0
2,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334,401011001401021003,838.0,82.0,...,947.0,2786.0,37.0,14.0,14.057,401011001,17265.0,5.373,401021003,24830.0
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004,430.0,13.0,...,1034.0,747.0,0.0,33.0,14.057,401011001,17265.0,14.156,401021004,28246.0
4,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278,401011001401021005,517.0,13.0,...,1107.0,1170.0,0.0,33.0,14.057,401011001,17265.0,3.395,401021005,25031.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10668,2018-02-01T00:00:00.000+11:00,404031105,4,404031109,2304,5876916,12172,404031109404031105,59.0,0.0,...,76.0,0.0,0.0,0.0,3.909,404031109,28657.0,1.867,404031105,26115.0
10669,2018-02-01T00:00:00.000+11:00,404031106,4,404031109,2967,5413932,16935,404031109404031106,105.0,0.0,...,151.0,0.0,0.0,1.0,3.909,404031109,28657.0,9.750,404031106,25913.0
10670,2018-02-01T00:00:00.000+11:00,404031107,4,404031109,1811,937583,8366,404031109404031107,262.0,0.0,...,142.0,0.0,0.0,18.0,3.909,404031109,28657.0,18.962,404031107,23052.0
10671,2018-02-01T00:00:00.000+11:00,404031108,4,404031109,2464,1334692,11329,404031109404031108,182.0,0.0,...,221.0,0.0,0.0,21.0,3.909,404031109,28657.0,13.439,404031108,23003.0


In [31]:
# augment the travel time and distance information to flow_adelaide_df
flow_adelaide_df['od_duration_value']=0.0 
flow_adelaide_df['od_distance_value']=0.0 

for idx in range(flow_adelaide_df.shape[0]):
    if idx%100 == 0:
        print(idx)

    # idx is the index in flow_adelaide_df
    origin_sa2_idx = flow_adelaide_df.loc[idx,'origin']
    destination_sa2_idx = flow_adelaide_df.loc[idx,'destination']
    
    # return the corresponding idx from OD_Google_API
    filter_idx = np.multiply(OD_google_with_map.loc[:, 'o_sa2_idx'] == origin_sa2_idx,
                             OD_google_with_map.loc[:, 'd_sa2_idx'] == destination_sa2_idx)
    idx_google_api = OD_google_with_map.index[filter_idx].tolist()[0] # this is the index in OD_google_with_map

    # 
    flow_adelaide_df.loc[idx, 'od_duration_value'] = OD_google_with_map.loc[idx_google_api, 'od_duration_value']
    flow_adelaide_df.loc[idx, 'od_distance_value'] = OD_google_with_map.loc[idx_google_api, 'od_distance_value']

# flow_adelaide_df.columns


0


/home/jasonllu/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600


In [32]:
# replace 0.0 values by 1.0
cols = ['sum_stay_duration','unique_agents','total_stays',
        'class_ART', 'class_BUS', 'class_COLL', 'class_FREE', 'class_HWY', 'class_LOCL', 
        'class_SUBA', 'class_TRK2', 'class_TRK4', 'class_UND',
        'od_duration_value', 'od_distance_value']

for col in cols:
    flow_adelaide_df.loc[flow_adelaide_df.loc[:,col] == 0.0, col] = 1.0


In [33]:
# dropped 433 observations. The df has nan.
flow_adelaide_df.dropna(how = 'any', inplace = True)


In [34]:
# add total road count as a variable
flow_adelaide_df['road_counts'] = np.sum(flow_adelaide_df[['class_ART', 'class_BUS', 'class_COLL', 'class_FREE', 'class_HWY', 'class_LOCL', 
                                         'class_SUBA', 'class_TRK2', 'class_TRK4', 'class_UND']], axis = 1)


In [35]:
flow_adelaide_df

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD,class_ART,class_BUS,...,class_UND,num_jobs_000_persons_origin,sa2_code16_x,median_income_per_job_aud_origin,num_jobs_000_persons_destination,sa2_code16_y,median_income_per_job_aud_destination,od_duration_value,od_distance_value,road_counts
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001,208.0,13.0,...,10.0,14.057,401011001,17265.0,14.057,401011001,17265.0,1.0,1.0,2227.0
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002,267.0,16.0,...,11.0,14.057,401011001,17265.0,5.459,401011002,24773.0,565.0,2897.0,2867.0
2,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334,401011001401021003,838.0,82.0,...,14.0,14.057,401011001,17265.0,5.373,401021003,24830.0,3293.0,39614.0,10415.0
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004,430.0,13.0,...,33.0,14.057,401011001,17265.0,14.156,401021004,28246.0,1603.0,20319.0,6590.0
4,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278,401011001401021005,517.0,13.0,...,33.0,14.057,401011001,17265.0,3.395,401021005,25031.0,2156.0,34698.0,7684.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10668,2018-02-01T00:00:00.000+11:00,404031105,4,404031109,2304,5876916,12172,404031109404031105,59.0,1.0,...,1.0,3.909,404031109,28657.0,1.867,404031105,26115.0,345.0,3289.0,520.0
10669,2018-02-01T00:00:00.000+11:00,404031106,4,404031109,2967,5413932,16935,404031109404031106,105.0,1.0,...,1.0,3.909,404031109,28657.0,9.750,404031106,25913.0,528.0,5066.0,1025.0
10670,2018-02-01T00:00:00.000+11:00,404031107,4,404031109,1811,937583,8366,404031109404031107,262.0,1.0,...,18.0,3.909,404031109,28657.0,18.962,404031107,23052.0,572.0,6601.0,1992.0
10671,2018-02-01T00:00:00.000+11:00,404031108,4,404031109,2464,1334692,11329,404031109404031108,182.0,1.0,...,21.0,3.909,404031109,28657.0,13.439,404031108,23003.0,695.0,6931.0,1849.0


In [36]:
flow_adelaide_df.columns

Index(['timestamp', 'destination', 'state', 'origin', 'unique_agents',
       'sum_stay_duration', 'total_stays', 'OD', 'class_ART', 'class_BUS',
       'class_COLL', 'class_FREE', 'class_HWY', 'class_LOCL', 'class_SUBA',
       'class_TRK2', 'class_TRK4', 'class_UND', 'num_jobs_000_persons_origin',
       'sa2_code16_x', 'median_income_per_job_aud_origin',
       'num_jobs_000_persons_destination', 'sa2_code16_y',
       'median_income_per_job_aud_destination', 'od_duration_value',
       'od_distance_value', 'road_counts'],
      dtype='object')

In [37]:
# save edge specific data
flow_adelaide_df.to_pickle('../../data_process/edge_specific_df.pickle')


# Prepare SA2 shapefile data

In [38]:
# origin and destination flow counts
origin_flow_counts = flow_adelaide_df.groupby(by="origin",as_index=False,sort=False).sum()[['origin','unique_agents','sum_stay_duration','total_stays']]
destination_flow_counts = flow_adelaide_df.groupby(by="destination",as_index=False,sort=False).sum()[['destination','unique_agents','sum_stay_duration','total_stays']]


In [39]:
# compute origin and destination entropy (w.r.t. location). flow location diversity.
# origin
origin_flow_count_n = flow_adelaide_df.groupby('origin')[['unique_agents','sum_stay_duration','total_stays']].transform('sum')
values = flow_adelaide_df[['unique_agents','sum_stay_duration','total_stays']]/origin_flow_count_n
flow_adelaide_df[['unique_agents_origin_entropy','sum_stay_duration_origin_entropy','total_stays_origin_entropy']] = \
    -(values*np.log(values))
origin_flow_entropy=flow_adelaide_df.groupby('origin',as_index=False,sort=False)[['unique_agents_origin_entropy','sum_stay_duration_origin_entropy','total_stays_origin_entropy']].sum()

# destination
destination_flow_count_n = flow_adelaide_df.groupby('destination')[['unique_agents','sum_stay_duration','total_stays']].transform('sum')
values = flow_adelaide_df[['unique_agents','sum_stay_duration','total_stays']]/destination_flow_count_n
flow_adelaide_df[['unique_agents_destination_entropy','sum_stay_duration_destination_entropy','total_stays_destination_entropy']] = \
    -(values*np.log(values))
destination_flow_entropy=flow_adelaide_df.groupby('destination',as_index=False,sort=False)[['unique_agents_destination_entropy','sum_stay_duration_destination_entropy','total_stays_destination_entropy']].sum()


In [40]:
origin_flow_counts

,origin,unique_agents,sum_stay_duration,total_stays
0,401011001,138443,357289619,1129284
1,401011002,72333,165959994,505142
2,401021003,59852,143401034,379786
3,401021004,156545,364372642,1066140
4,401021005,37564,91767719,282994
...,...,...,...,...
98,404031105,22346,49982722,152818
99,404031106,139940,286329047,863431
100,404031107,250037,557070490,1670710
101,404031108,169390,377419921,1167121


In [41]:
# merge data to sa2_adelaide_road_proj
# augment income and jobs
sa2_data_prep=pd.merge(sa2_adelaide_road_proj, jobs_all_sub, left_on='SA2_MAIN16', right_on='sa2_code16', how = 'inner')
sa2_data_prep=pd.merge(sa2_data_prep, origin_flow_counts, left_on='SA2_MAIN16', right_on='origin', how='inner', suffixes=[None,'_origin_counts'])
sa2_data_prep=pd.merge(sa2_data_prep, destination_flow_counts, left_on='SA2_MAIN16', right_on='destination', how='inner', suffixes=[None,'_destination_counts'])
sa2_data_prep=pd.merge(sa2_data_prep, origin_flow_entropy, left_on='SA2_MAIN16', right_on='origin', how='inner')
sa2_data_prep=pd.merge(sa2_data_prep, destination_flow_entropy, left_on='SA2_MAIN16', right_on='destination', how='inner')

# rename the '_origin_counts'
sa2_data_prep = sa2_data_prep.rename(columns={'unique_agents':'unique_agents_origin_counts',
                                      'sum_stay_duration':'sum_stay_duration_origin_counts',
                                      'total_stays':'total_stays_origin_counts'})

sa2_data_prep.columns


Index(['SA2_MAIN16', 'SA2_5DIG16', 'SA2_NAME16', 'SA3_CODE16', 'SA3_NAME16',
       'SA4_CODE16', 'SA4_NAME16', 'GCC_CODE16', 'GCC_NAME16', 'STE_CODE16',
       'STE_NAME16', 'AREASQKM16', 'class_ART', 'class_BUS', 'class_COLL',
       'class_FREE', 'class_HWY', 'class_LOCL', 'class_SUBA', 'class_TRK2',
       'class_TRK4', 'class_UND', 'geometry', 'num_jobs_000_persons',
       'sa2_code16', 'median_income_per_job_aud_persons', 'origin_x',
       'unique_agents_origin_counts', 'sum_stay_duration_origin_counts',
       'total_stays_origin_counts', 'destination_x',
       'unique_agents_destination_counts',
       'sum_stay_duration_destination_counts',
       'total_stays_destination_counts', 'origin_y',
       'unique_agents_origin_entropy', 'sum_stay_duration_origin_entropy',
       'total_stays_origin_entropy', 'destination_y',
       'unique_agents_destination_entropy',
       'sum_stay_duration_destination_entropy',
       'total_stays_destination_entropy'],
      dtype='object')

In [42]:
# save 
sa2_data_prep.to_pickle('../../data_process/node_specific_df.pickle')
